### Loading the model

In [10]:
import sys
sys.path.insert(0, '../')

from transformers import BertConfig
from model.model import MidiBert
import pickle
import torch


print("Loading Dictionary")
with open('/root/code/HugoA45/music_project/music_project/model/CP.pkl', 'rb') as f:
    e2w, w2e = pickle.load(f)

# Define the configuration for the BERT model
configuration = BertConfig(
    max_position_embeddings=512,
    position_embedding_type='relative_key_query',
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=12
)

# Initialize the model
midibert = MidiBert(bertConfig=configuration, e2w=e2w, w2e=w2e)

# Define the path to your checkpoint here
ckpt_path = '/root/code/HugoA45/music_project/music_project/model/pretrain_model.ckpt'

# Load the checkpoint
checkpoint = torch.load(ckpt_path, map_location='cpu')

# Remove the unexpected key from the state dictionary
if "bert.embeddings.position_ids" in checkpoint['state_dict']:
    embeddings_position_ids =  checkpoint['state_dict']["bert.embeddings.position_ids"]
    del checkpoint['state_dict']["bert.embeddings.position_ids"]

# Load the state dictionary from the checkpoint into the model
midibert.load_state_dict(checkpoint['state_dict'])


Loading Dictionary


<All keys matched successfully>

In [2]:
checkpoint.keys()


dict_keys(['epoch', 'state_dict', 'best_acc', 'valid_acc', 'valid_loss', 'train_loss', 'optimizer'])

### Bert model

In [11]:
midibert.evasl()


MidiBert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (distance_embedding): Embedding(1023, 64)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Layer

In [4]:
embeddings_position_ids.shape


torch.Size([1, 512])

### Number of parameters

In [5]:
model = midibert.bert.from_pretrained('bert-base-uncased')
num_parameters = sum(p.numel() for p in model.parameters())

print(f'The model has {num_parameters} parameters.')


AttributeError: '_IncompatibleKeys' object has no attribute 'bert'

 In a typical BERT model, the state_dict keys for the transformer layers usually follow this pattern:

bert.encoder.layer.{i}.{sub_layer}.{param}
where:

{i} is the layer index (from 0 to 11 for a 12-layer BERT model),
{sub_layer} is the sub-layer type, which can be attention.self, attention.output, intermediate, or output,
{param} is the parameter type, which can be weight or bias for linear layers, and gamma or beta for layer normalization.
To count the number of unique transformer layers in the state_dict, you can extract the layer index from each key and add it to a set (which automatically removes duplicates), then count the number of elements in the set.

In [ ]:
state_dict = checkpoint['state_dict']

# Initialize an empty set to store the layer indices
layer_indices = set()

# Iterate over the keys in the state dictionary
for key in state_dict.keys():
    # Split the key into parts
    parts = key.split('.')
    # If this key corresponds to a transformer layer
    if parts[0] == 'bert' and parts[1] == 'encoder' and parts[2] == 'layer':
        # Extract the layer index and add it to the set
        layer_indices.add(int(parts[3]))

# Print the number of unique layers
print(f"The model has {len(layer_indices)} unique transformer layers.")


The model has 12 unique transformer layers.


In [ ]:
# Get the names of the layers
layer_names = state_dict.keys()

# Get the unique keys from the layers
unique_keys = set()
for name in layer_names:
    # Split the name by '.' and get the first part
    key = name.split('.')[0]
    unique_keys.add(key)

# Print the unique keys
for key in unique_keys:
    print(key)


word_emb
in_linear
bert


the state_dict keys for the transformer layers usually follow this pattern:

bert.encoder.layer.{i}.{sub_layer}.{param}
where:

{i} is the layer index (from 0 to 11 for a 12-layer BERT model),
{sub_layer} is the sub-layer type, which can be attention.self, attention.output, intermediate, or output,
{param} is the parameter type, which can be weight or bias for linear layers, and gamma or beta for layer normalization.
To count the number of unique transformer layers in the state_dict, you can extract the layer index from each key and add it to a set (which automatically removes duplicates), then count the number of elements in the set.

e2w

In [ ]:
e2w.keys()


dict_keys(['Bar', 'Position', 'Pitch', 'Duration'])

In [ ]:
w2e['Bar'], w2e['Position'], w2e['Pitch'], w2e['Duration']


({0: 'Bar New', 1: 'Bar Continue', 2: 'Bar <PAD>', 3: 'Bar <MASK>'},
 {0: 'Position 1/16',
  1: 'Position 2/16',
  2: 'Position 3/16',
  3: 'Position 4/16',
  4: 'Position 5/16',
  5: 'Position 6/16',
  6: 'Position 7/16',
  7: 'Position 8/16',
  8: 'Position 9/16',
  9: 'Position 10/16',
  10: 'Position 11/16',
  11: 'Position 12/16',
  12: 'Position 13/16',
  13: 'Position 14/16',
  14: 'Position 15/16',
  15: 'Position 16/16',
  16: 'Position <PAD>',
  17: 'Position <MASK>'},
 {0: 'Pitch 22',
  1: 'Pitch 23',
  2: 'Pitch 24',
  3: 'Pitch 25',
  4: 'Pitch 26',
  5: 'Pitch 27',
  6: 'Pitch 28',
  7: 'Pitch 29',
  8: 'Pitch 30',
  9: 'Pitch 31',
  10: 'Pitch 32',
  11: 'Pitch 33',
  12: 'Pitch 34',
  13: 'Pitch 35',
  14: 'Pitch 36',
  15: 'Pitch 37',
  16: 'Pitch 38',
  17: 'Pitch 39',
  18: 'Pitch 40',
  19: 'Pitch 41',
  20: 'Pitch 42',
  21: 'Pitch 43',
  22: 'Pitch 44',
  23: 'Pitch 45',
  24: 'Pitch 46',
  25: 'Pitch 47',
  26: 'Pitch 48',
  27: 'Pitch 49',
  28: 'Pitch 50',
  29